In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import numpy as np
from typing import Dict
import polars as pl

from src.common.AssetData import AssetData
from src.common.AssetDataPolars import AssetDataPolars
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.featureAlchemy.FeatureFinancialData import FeatureFinancialData


In [3]:
assets=AssetFileInOut("../src/stockGroups/bin").loadDictFromFile("group_debug")

# Convert to Polars for speedup
assetspl: Dict[str, AssetDataPolars] = {}
for ticker, asset in assets.items():
    assetspl[ticker] = AssetDataService.to_polars(asset)
    
ticker = "AAPL"
asset = assetspl[ticker]

fin_quar = asset.financials_quarterly
fin_ann = asset.financials_annually
asset_about = asset.about

In [4]:
print(fin_quar.columns)
params = {
    "timesteps": 50,
}
ffd = FeatureFinancialData(asset, [1,2,3,5,9,21,63,121,210], params=params)
print(asset.shareprice.columns)
asShare = ffd.shareprice
finQuarJoin = ffd.fin_quar_join

['fiscalDateEnding', 'reportedDate', 'reportedEPS', 'estimatedEPS', 'surprise', 'surprisePercentage', 'reportTime', 'grossProfit', 'totalRevenue', 'ebit', 'ebitda', 'totalAssets', 'totalCurrentLiabilities', 'totalShareholderEquity', 'commonStockSharesOutstanding', 'operatingCashflow']
['Date', 'Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume', 'Dividends', 'Splits']


In [5]:
asShare["daysToReport"].item(2000)

0.0

In [6]:
featureNames = ffd.getFeatureNames()
features = ffd.apply(pd.Timestamp(2025, 1, 23, tz = "UTC").date(), 1)

In [7]:
featureNamesTimeseries = ffd.getTimeFeatureNames()
featuresTimeseries = ffd.apply_timeseries(pd.Timestamp(2025, 1, 23, tz = "UTC").date())